In [ ]:
import glob
import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
filenames=glob.glob(r"E:\aeromodelismo\MODELO_3D_SANTIAGO_NOROESTE/*/f*.obj")

In [ ]:
X=[]
Y=[]
Z=[]
res=[]
for file in tqdm.tqdm(filenames):
    with open(file,"r") as f:
        lines=f.readlines()
    for line in lines:
        if line[:2]=="v ":
            res.append(line.split()+list([file]))

In [ ]:
df_coords=pd.DataFrame(res)
df_coords[3]=df_coords[3].astype("float64")
df_coords[1]=df_coords[1].astype("float64")
df_coords[2]=df_coords[2].astype("float64")
df_coords=df_coords.sort_values(by=[3]).reset_index(drop=True)

In [ ]:
df_coords=df_coords.drop_duplicates(subset=[1,2,3]).reset_index(drop=True)
df_coords=df_coords.astype({1:"int",2:int,3:int})
df_coords

In [ ]:
new_image_width = int(np.max(df_coords[1]))+1
new_image_height = int(np.max(df_coords[2]))+1
new_image_height,new_image_width

In [ ]:
color = (0,0,0)
result = np.full((new_image_height,new_image_width, 3), color, dtype=np.uint8)
result=result[:,:,0]


In [ ]:
import tqdm
for i in tqdm.tqdm(range(len(df_coords))):
    result[df_coords.loc[i,2],df_coords.loc[i,1]]=df_coords.loc[i,3]
    

In [5]:
with open("E:/aeromodelismo/MODELO_3D_SANTIAGO_NOROESTE/d000/doc.kml","r") as kml:
    li=kml.readlines()
lineas_utiles=[]
for l in li:
    l=l.replace("\n","")
    lineas_utiles.append(l)
    if l.find("href")>=0:
        break
lineas_utiles

['<?xml version="1.0" encoding="UTF-8"?>',
 '<kml xmlns="http://www.opengis.net/kml/2.2" hint="quadtree">',
 '  <Folder>',
 '    <Region>',
 '      <LatLonAltBox>',
 '        <west>-99.782989391660777</west>',
 '        <east>-99.780128777155554</east>',
 '        <south>19.788728569098481</south>',
 '        <north>19.792097638528556</north>',
 '        <minAltitude>2540.1094647646382</minAltitude>',
 '        <maxAltitude>2593.080295705453</maxAltitude>',
 '        <altitudeMode>absolute</altitudeMode>',
 '      </LatLonAltBox>',
 '      <Lod>',
 '        <minLodPixels>256</minLodPixels>',
 '        <maxLodPixels>512</maxLodPixels>',
 '      </Lod>',
 '    </Region>',
 '    <Placemark>',
 '      <Model>',
 '        <altitudeMode>absolute</altitudeMode>',
 '        <Location>',
 '          <longitude>-99.784036190795319</longitude>',
 '          <latitude>19.788399159150078</latitude>',
 '          <altitude>2531.0369093063096</altitude>',
 '        </Location>',
 '        <Link>',
 '

In [ ]:
metrosxpixel longitud de array/ diferencia entre west y east en crs:3857

In [ ]:
import rasterio
from affine import Affine
##extremo superior izquierdo

latitud=-11107562.4
longitud=2249011.4
metrosxpixel=1
crs=3857
matriz=result


transform=Affine(metrosxpixel,0,latitud,0,-metrosxpixel,longitud) ## 0 representa las componetes de direccion de la coordenada
out_meta=({"driver": "GTiff",
            "height": matriz.shape[0],
            "width": matriz.shape[1],
            "count":1,
            "dtype":np.float32,
            "crs":3857,
          'transform':transform})
with rasterio.open(r"E:\geoshapes\blac1k.tif",'w', **out_meta) as dest_file:
    dest_file.write(matriz, 1)